In [1]:
import gym
from gym import Wrapper
import numpy as np
import math
import pygame
import time
import random

In [18]:
!pip install pygad

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 KB 161.9 kB/s eta 0:00:001m201.8 kB/s eta 0:00:01


# Policy Iteration 
here we are going to implement Policy Iteration in Forzen_Lake (one of the environmnet in OpenAi Gym library .
![title](Policy_itr.png)

In [2]:
def policy_iteration(P, nS, nA,env ,  gamma=0.9, tol=1e-4):
    '''
    parameters:
        P: transition probability matrix
        nS: number of states
        nA: number of actions
        gamma: discount factor
        tol: tolerance for convergence
    returns:
        value_function: value function for each state
        policy: policy for each state
    '''
    # initialize value function and policy
    
    value_function = np.zeros(nS)
    Policy = []
    Possible_actions = [0 , 1 , 2 , 3]
    '''
    action 0 : Move to the right
    acrion 1 : Move to the left
    action 2 : Move up
    actino 3 : Move down

    '''
    for i in range(nS) :
#         in this loop we add an action list to Policy list , we cinsider one possible_action list for each state
        Policy.append(Possible_actions)
    stable_policy = True
    counter = 0 
#     while stable_policy == True :
    for i in range(1) :
        value_function = Policy_evaluation(Policy , env , nS  , gamma , value_function, theta = 0.01 )  
        q = value_function
        stable_policy , Policy = Policy_Improvemnet(Policy , env , nS  , gamma , q, teta = 0.01)
        
    # Implement policy iteration here #
    return value_function, Policy

In [9]:
def Policy_evaluation(policy , env , nS  , gamma , value_function, theta = 0.01 ):


    counter = 0 
    
#      for i in range(64):
    while True :
        delta = -math.inf
        values = np.zeros(nS)
        for state in range(nS) :
            if state == 5 or state == 7 or state == 11 or state == 12  :
                values[state] =0
            elif  state == 15:
                values[state] = 0

            else :
                new_value = 0
                number_of_actions = len(policy[state])
                for action in policy[state] :
                    for a in env.env.P[state][action]:
                        reward = 0
                # Calculate the expected value
                        if a[3] == True and a[1]== 15:
                            reward = 1
                        elif a[3] == True :
                             reward = -2
                        elif a[3] == False :
                            reward =0
                    

                # Calculate the expected value
                        new_value += a[0]*(reward + gamma * value_function[a[1]])/number_of_actions
                    
                values[state] = format(new_value, ".3f")
            
                delta = max(delta, np.abs( value_function[state] - values[state]))
            
        value_function = values
        
        counter +=1 
           

        if delta < theta :
                break 
    return value_function
            
    
    
    

In [8]:
def Policy_Improvemnet(Policy , env , nS  , gamma , value_function, teta = 0.01) :
    policy_stable = True
    updated_Policy = []
    for state in range(nS) :
        max_value = - math.inf
        
        actions = []
        quality_of_actions = []
        
        new_policy = []
        number_of_actions = len(Policy[state])

        for action in Policy[state] :
            new_value = 0 
            counter = 0
            for a in env.env.P[state][action]:
                    reward = 0
                # Calculate the expected value
                
                    if a[3] == True and a[1]== 15:
                        reward = 1
                    elif a[3] == True :
                        reward = -2
                    elif a[3]==False :
                        reward = 0
                    new_value+= a[0]*(reward + gamma * value_function[a[1]])/number_of_actions
            if max_value < new_value :
                    max_value = new_value
            actions.append(action)
            quality_of_actions.append( new_value)
            
        for i in range(len(actions)):
                if max_value - 0.0001 < quality_of_actions[i] :
                    new_policy.append(actions[i])
        updated_Policy.append(new_policy)
        
    if updated_Policy == Policy :
            
        policy_stable = False
                    
           
                    
            
    return policy_stable , updated_Policy

In [96]:
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        result = self.env.step(action)
        # modify the reward based on some co111ndition
        if result[1] == 1:
            result[1] = 1
        elif result[1] == 0:
            result[1] = -1
        else:
            result[1] = -2
        return result

    def reset(self):
        return self.env.reset()

# # create the FrozenLake environment
# env = gym.make('FrozenLake-v0')

# # create the reward wrapper
# wrapped_env = CustomRewardWrapper(env)

# # use the wrapped environment for training

In [6]:
def Operation_of_agent(Policy) :
#     in this function we run the environmenet with given policy
    env = gym.make("FrozenLake-v1", is_slippery=False , render_mode='human' )
    # run a random agent in the environment
    obs = env.reset()
    done = False
    action = random.choice(Policy[0])
    while True :
        result  = env.step(action)
        action = random.choice(Policy[result[0]])
        print(result[0])
        print(result[2])
        
        if result[2] == True:
            break

    
    env.close()

Checking the results :\
First we adjust the discount factor :

In [11]:
# discount factor = 1
new_env = gym.make("FrozenLake-v1", is_slippery=False )
# new_env = CustomRewardWrapper(env)
state = new_env.reset()
pi , po = policy_iteration(new_env.P , new_env.observation_space.n , new_env.action_space.n , new_env,gamma =0.9  )
print(po)

print(pi)
Operation_of_agent(po)

env.close()

[[0, 3], [0], [3], [0], [3], [0, 1, 2, 3], [1], [0, 1, 2, 3], [2], [1], [1], [0, 1, 2, 3], [0, 1, 2, 3], [2], [2], [0, 1, 2, 3]]
[-1.088 -1.315 -1.237 -1.41  -1.373  0.    -1.555  0.    -1.435 -1.357
 -1.236  0.     0.    -1.144 -0.367  0.   ]
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False
0
False


KeyboardInterrupt: 

In [12]:
print(env.P)

{0: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 4, 0.0, False)], 2: [(1.0, 1, 0.0, False)], 3: [(1.0, 0, 0.0, False)]}, 1: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 5, 0.0, True)], 2: [(1.0, 2, 0.0, False)], 3: [(1.0, 1, 0.0, False)]}, 2: {0: [(1.0, 1, 0.0, False)], 1: [(1.0, 6, 0.0, False)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 2, 0.0, False)]}, 3: {0: [(1.0, 2, 0.0, False)], 1: [(1.0, 7, 0.0, True)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 3, 0.0, False)]}, 4: {0: [(1.0, 4, 0.0, False)], 1: [(1.0, 8, 0.0, False)], 2: [(1.0, 5, 0.0, True)], 3: [(1.0, 0, 0.0, False)]}, 5: {0: [(1.0, 5, 0, True)], 1: [(1.0, 5, 0, True)], 2: [(1.0, 5, 0, True)], 3: [(1.0, 5, 0, True)]}, 6: {0: [(1.0, 5, 0.0, True)], 1: [(1.0, 10, 0.0, False)], 2: [(1.0, 7, 0.0, True)], 3: [(1.0, 2, 0.0, False)]}, 7: {0: [(1.0, 7, 0, True)], 1: [(1.0, 7, 0, True)], 2: [(1.0, 7, 0, True)], 3: [(1.0, 7, 0, True)]}, 8: {0: [(1.0, 8, 0.0, False)], 1: [(1.0, 12, 0.0, True)], 2: [(1.0, 9, 0.0, False)], 3: [(1.0, 4, 0.0, False)]}, 9: {0: [(1